# Efficient `pandas`

In [ ]:
%pylab inline
plt.style.use("bmh")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import string

# Data

In [ ]:
df = pd.DataFrame(np.arange(2000).reshape((1000,2)),
                  columns=['a', 'b'],
                  index=np.random.choice(list(string.ascii_lowercase), 1000, replace=True))

In [ ]:
df.head()

# Loops

## Naive

In [ ]:
def iterate_df(df):
    """Iterate over df in Python loop."""

    result = []
    
    for i in range(df.shape[0]):
        row = df.iloc[i]
        result.append(row['a']/row['b'])
    return pd.Series(result, name="div_result", index=df.index)

In [ ]:
%timeit -n 10 -r 5 iterate_df(df)

## Using `iterrows`

In [ ]:
def iterate_df_rows(df):
    """Iterate over df in Python loop."""

    result = []
    
    for ri, row in df.iterrows():
        result.append(row['a']/row['b'])
    return pd.Series(result, name="AgeGroup", index=df.index)

In [ ]:
%timeit -n 20 -r 5 iterate_df_rows(df)

## Using `apply`

In [ ]:
%timeit -n 20 -r 5 df.apply(lambda x: x['a']/x['b'], axis=1)

## Using vectorization

In [ ]:
%timeit -n 20 -r 5 df['a']/df['b']

# Memory

In [ ]:
titanic_train = pd.read_csv("../W3/data/train.csv", index_col="PassengerId")
titanic_test = pd.read_csv("../W3/data/test.csv", index_col="PassengerId")
titanic = pd.concat([titanic_train, titanic_test], sort=False)

titanic.head(5)

In [ ]:
titanic.dtypes

In [ ]:
titanic.info(memory_usage="deep")

In [ ]:
titanic["Pclass"] = pd.to_numeric(titanic["Pclass"], downcast="unsigned")

In [ ]:
titanic["SibSp"] = pd.to_numeric(titanic["SibSp"], downcast="unsigned")
titanic["Parch"] = pd.to_numeric(titanic["SibSp"], downcast="unsigned")

In [ ]:
titanic.info(memory_usage="deep")

## How to read it with correct `dtype`s right away?

In [ ]:
titanic_train_trunc = pd.read_csv("../W3/data/train.csv", index_col="PassengerId",
                                  dtype={"Pclass":np.uint8,
                                         "SibSp":np.uint8,
                                         "Parch":np.uint8,
                                         "Survived":np.float32,
                                         "Age":np.float32,
                                         "Fare":np.float32},
                                  converters={"Sex": lambda x: (np.uint8(1)
                                                                if x=="female" else np.uint8(0)),
                                              "Embarked": lambda x: 0 if x=="S" else (1 if x=="C" else 2)})
titanic_test_trunc = pd.read_csv("../W3/data/test.csv", index_col="PassengerId",
                                 dtype={"Pclass":np.uint8,
                                        "SibSp":np.uint8,
                                        "Parch":np.uint8,
                                        "Survived":np.float32,
                                        "Age":np.float32,
                                        "Fare":np.float32},
                                 converters={"Sex": lambda x: np.uint8(1) if x=="female" else np.uint8(0),
                                             "Embarked": lambda x: 0 if x=="S" else (1 if x=="C" else 2)})
titanic_trunc = pd.concat([titanic_train_trunc, titanic_test_trunc], sort=False)

titanic_trunc.info(memory_usage="deep")

In [ ]:
titanic_trunc.select_dtypes(np.uint8).head()

## `eval` and `query`

In [ ]:
%load_ext memory_profiler

In [ ]:
%%writefile evals.py
import pandas as pd

def direct_computation(df):
    result = df['b']/df['a'] + df['a']/df['b']
    return result

def eval_computation(df):
    result = pd.eval("df.b/df.a + df.a/df.b")
    return result

In [ ]:
from evals import direct_computation, eval_computation

In [ ]:
df = pd.DataFrame(np.arange(20000000).reshape((10000000,2)),
                  columns=['a', 'b'],
                  index=np.random.choice(list(string.ascii_lowercase), 10000000, replace=True))

In [ ]:
df.info(memory_usage="deep")

In [ ]:
%mprun -f direct_computation direct_computation(df)

In [ ]:
%mprun -f eval_computation eval_computation(df)

In [ ]:
%timeit -n 10 -r 3 direct_computation(df)

In [ ]:
%timeit -n 10 -r 3 eval_computation(df)

In [ ]:
%%writefile complex_evals.py
import pandas as pd
import numpy as np
def direct_computation(df):
    result = np.cos(df['b']) + np.sin(df['a'])
    return result

def eval_computation(df):
    result = pd.eval("cos(df.b) + sin(df.a)")
    return result

In [ ]:
from complex_evals import direct_computation, eval_computation

In [ ]:
%mprun -f direct_computation direct_computation(df)

In [ ]:
%mprun -f eval_computation eval_computation(df)

In [ ]:
%timeit -n 10 -r 3 direct_computation(df)

In [ ]:
%timeit -n 10 -r 3 eval_computation(df)

In [ ]:
%%writefile query.py
import pandas as pd

def direct_query(df):
    return df[(df['a'] > 7) & (df['b'] < 35)]

def eval_query(df):
    return df.query("a > 7 and b < 35")

In [ ]:
from query import direct_query, eval_query

In [ ]:
%mprun -f direct_query direct_query(df)

In [ ]:
%mprun -f eval_query eval_query(df)

In [ ]:
%timeit df[(df['a'] > 7) & (df['b'] < 35)]

In [ ]:
%timeit df.query("a > 7 and b < 35")